In [173]:
from google.cloud import bigquery
import pandas as pd
import pandas as pd
import numpy as np
from newspaper import Article
import nltk
import pickle

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity 
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

import matplotlib.pyplot as plt
import string
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from gdelt_helper import ProcessPipeline, get_text
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Flatten, Embedding,BatchNormalization, Activation, ZeroPadding2D
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.python.keras.layers.convolutional import UpSampling2D, Conv2D
from tensorflow.python.keras.models import Sequential, Model, load_model
from sklearn.metrics import accuracy_score
import urllib.request
import zipfile
import glove.Corpus
from glove import Corpus, Glove

ModuleNotFoundError: No module named 'glove.Corpus'

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sizhenhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sizhenhan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/sizhenhan/Downloads/direct-blend-250919-8c595ee2d9b5.json"
client = bigquery.Client()
rawQuery = client.query("""
SELECT DATE,THEMES,DocumentIdentifier \
FROM `gdelt-bq.gdeltv2.gkg` \
WHERE DATE>=20181231000000 AND DATE<20190107000000 \
AND Locations like "%US%" \
AND THEMES like "%ENV_GREEN%" """)
rawDf = rawQuery.to_dataframe()

In [3]:
rawDf = pd.read_csv("/Users/sizhenhan/Downloads/NLP-analysis-of-Energy-sector-based-on-Gdelt-master/outputs/step1_bigquery_output.csv")

In [4]:
rawDf.head()

,DATE,THEMES,DocumentIdentifier
0,20190101060000,EDUCATION;SOC_POINTSOFINTEREST;SOC_POINTSOFINT...,https://www.daijiworld.com/chan/exclusiveDispl...
1,20190101061500,TAX_FNCACT;TAX_FNCACT_MAN;ARREST;SOC_GENERALCR...,https://caymannewsservice.com/2018/12/
2,20190101063000,TAX_FNCACT;TAX_FNCACT_LEADER;ENV_NUCLEARPOWER;...,https://www.vesti.bg/tehnologii/bil-gejts-sash...
3,20190101061500,ENV_GREEN;WB_507_ENERGY_AND_EXTRACTIVES;WB_525...,https://www.ajc.com/business/economy/georgia-p...
4,20190101061500,ENV_GREEN;WB_507_ENERGY_AND_EXTRACTIVES;WB_525...,https://pv-magazine-usa.com/2018/12/18/breakin...


In [5]:
len(rawDf)

2988

In [22]:
texts = list(map(lambda x:get_text(x),rawDf.DocumentIdentifier))

Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from http://www.windermeresun.com/2018/12/31/the-shift-toward-clean-power/, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
fail to download news from https://www.kansas.com/entertainment/celebrities/article223796160.html, it is outdated and 404 error
fail to download news from https://www.sunherald.com/news/business/article223796160.html, it is outdated and 404 error
Succeed!
fail to download news from https://www.beaumontenterprise.com/news/education/article/The-Latest-New-Mexico-swears-in-Democratic-13501599.php, it is outdated and 404 error
fail to download news from https://www.wral.com/democrat-lujan-grisham-takes-

/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 38 bytes but only got 0. Skipping tag 11
  " Skipping tag %s" % (size, len(data), tag)
/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 38 bytes but only got 0. Skipping tag 305
  " Skipping tag %s" % (size, len(data), tag)
/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 0. Skipping tag 306
  " Skipping tag %s" % (size, len(data), tag)
/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2060 bytes but only got 908. Skipping tag 59932
  " Skipping tag %s" % (size, len(data), tag)
/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlu

Succeed!
Succeed!
Succeed!
fail to download news from https://www.miamiherald.com/news/article223738125.html, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
fail to download news from https://www.argus-press.com/news/national/article_e243ebad-26cf-541d-923d-26b351112148.html, it is outdated and 404 error
fail to download news from https://www.tri-cityherald.com/news/business/national-business/article223772670.html, it is outdated and 404 error
fail to download news from https://solarindustrymag.com/texas-mmex-resources-makes-solar-moves/, it is outdated and 404 error
fail to download news from http://redgreenandblue.org/2018/12/31/100-us-cities-u-s-cities-committed-100-renewable-energy-can-basis-national-plan/, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
fail to download news from http://redgreenandblue.org/2018/12/31/coming-2019-floating-solar-us-department-energy-board/, it is outdated and 404 error
fail to download news from https://www.whig.com/article/20181231

Building prefix dict from /Users/sizhenhan/anaconda3/lib/python3.7/site-packages/jieba/dict.txt ...
Loading model from cache /var/folders/4l/n3ktgp0n0t5bqbyg715w_3380000gn/T/jieba.cache
Loading model cost 0.7646541595458984 seconds.
Prefix dict has been built succesfully.


Succeed!
fail to download news from https://www.kanw.com/post/democrat-lujan-grisham-takes-oath-new-mexico-governor, it is outdated and 404 error
Succeed!
Succeed!
fail to download news from https://www.greensboro.com/ap/us_world/tops-on-house-dems-to-do-list-try-to-end/article_62bb6eda-f557-5763-a584-0052b7fcd3a4.html, it is outdated and 404 error
fail to download news from https://www.wacotrib.com/news/ap_nation/politics/tops-on-house-dems-to-do-list-try-to-end/article_5bdd6ee3-4cd0-5a39-bbdc-296a7d9caf92.html, it is outdated and 404 error
fail to download news from https://www.dentonrc.com/ap/washington/tops-on-house-dems-to-do-list-try-to-end/article_d7352b13-89c7-52eb-9d35-5661c0e05e09.html, it is outdated and 404 error
fail to download news from https://www.hickoryrecord.com/news/us/ap/tops-on-house-dems-to-do-list-try-to-end/article_1e9559db-5d96-50e0-bf2b-73209db69894.html, it is outdated and 404 error
fail to download news from https://www.mdjonline.com/news/national/tops-on-h

...

Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from http://www.coast1009.com/politics/the-note-democrats-take-power-as-trumps-hand-grows-weaker/, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
fail to download news from http://bharatapress.com/2019/01/02/reviewing-renewable-energy-group-regi-green-plains-partners-gpp/, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from https://www.forbes.com/sites/bobeccles/2019/01/03/the-importance-of-the-extractive-minerals-processing-sector-to-the-sustainable-development-goals/, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from https://www.beaumontenterprise.com/news/article/EXCHANGE-Containers-to-office-space-for-firm-s-13508366.php, it is outdated and 404 error
Succeed!
Succeed!

fail to download news from https://www.stltoday.com/news/national/govt-and-politics/new-congress-gavels-more-diverse-ready-to-confront-trump/article_3cd4d26d-9c92-5873-b32a-5f945113cd4b.html, it is outdated and 404 error
Succeed!
fail to download news from https://www.pantagraph.com/news/national/govt-and-politics/new-congress-gavels-more-diverse-ready-to-confront-trump/article_115bb01b-14df-5d1d-ad19-dff41f365256.html, it is outdated and 404 error
Succeed!
fail to download news from https://wcfcourier.com/news/national/govt-and-politics/new-congress-gavels-more-diverse-ready-to-confront-trump/article_429d1d9d-9e17-5244-b6aa-d85ab7188ef5.html, it is outdated and 404 error
fail to download news from https://www.aspendailynews.com/associated_press/new-congress-gavels-more-diverse-ready-to-confront-trump/article_f1d6a6fb-610d-5c13-893a-57c319fdc534.html, it is outdated and 404 error
Succeed!
Succeed!
fail to download news from https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objec

fail to download news from https://www.greensboro.com/ap/us_world/diverse-new-congress-gavels-in-ready-to-confront-trump/article_0de6321d-788d-5125-a08c-6e18049b5313.html, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from https://www.msn.com/en-us/news/politics/diverse-new-congress-gavels-in-ready-to-confront-trump/ar-BBRLGJq, it is outdated and 404 error
fail to download news from http://mynorthwest.com/1232088/new-congress-gavels-in-with-nod-to-history-warning-to-trump/, it is outdated and 404 error
fail to download news from https://www.theintelligencer.com/news/article/New-Congress-gavels-in-with-nod-to-history-13505950.php, it is outdated and 404 error
Succeed!
fail to download news from https://dminute.com/2019/01/03/brunswick-bc-holder-impala-asset-management-has-decreased-its-position-as-share-price-declined-dte-energy-co-dte-shareholder-new-york-state-teachers-retirement-system-has-boosted-its-holding-by/, it is outda

fail to download news from https://www.eveningtelegraph.co.uk/2019/01/04/american-farm-show-unveils-500kg-butter-sculpture/, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
fail to download news from https://www.mysanantonio.com/news/texas/article/New-Mexico-reconsiders-permission-for-more-13508981.php, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from https://seekingalpha.com/article/4231525-renewable-energy-etfs-now-outperforming-will-continue-2019, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
fail to download news from https://www.forbes.com/sites/uhenergy/2019/01/04/policies-or-technology-the-key-to-a-sustainable-energy-future/, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
fail to download news from https://www.thepublicopinion.com/business/associated_press/global-transformers-market-for-renewable-energy---to-post/article_95c3c79d-de33-5e02-aecc-c3944b54e604.html, it is outdated and 404 error
Succeed!
Succeed!


Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from https://www.argus-press.com/news/national/article_69c1f3f8-1088-586a-9ca8-db42d3967278.html, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from https://www.environmentguru.com/pages/elements/element.aspx?id=9176825, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
fail to download news from http://www.ccenterdispatch.com/news/state/article_96c3a36e-cfdb-562a-8693-60bd9513486f.html, it is outdated and 404 error
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
fail to download news from https://www.normantranscript.com/cnhi_network/new-laws-are-putting-california-further-at-odds-with-trump/article_87

In [35]:
texts_new = []
for text in texts:
    if text is not None:
        texts_new.append(text)
len(texts_new)

2275

In [38]:
with open('texts_raw_new.pickle', 'wb') as handle:
    pickle.dump(texts_new, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('texts_raw_new.pickle', 'rb') as handle:
    texts = pickle.load(handle)

In [7]:
pipeline = ProcessPipeline(texts)

textsProcessed = pipeline.run()

language is en
language is en
language is bg
language is de
language is en
Not able to detect language
language is en
language is en
language is vi
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is es
language is en
language is en
language is en
language is en
language is en
language is en
language is de
language is ar
language is en
language is en
language is es
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is es
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is ko
language is es
language is ko
language is en
language is pl
language is en
language is en
language is en
language is en
language is es
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is 

language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
Not able to detect language
language is en
language is fr
language is ko
language is pt
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is sv
Not able to detect language
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
Not able to detect language
language is da
Not able to detect language
language is en
language is en
language is en
language is en
language is en
language is en
language is ru
language is zh-cn
language is ar
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is es
language is en
language is en


language is es
language is en
language is en
language is en
language is ko
language is de
language is en
language is en
language is en
language is es
language is en
language is en
language is en
language is de
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is es
Not able to detect language
language is en
language is en
language is en
language is en
language is en
language is de
language is he
language is en
language is en
language is en
language is en
language is en
language is en
language is en
Not able to detect language
language is es
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en

language is en
language is en
language is en
language is de
language is es
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is de
language is de
language is es
Not able to detect language
language is de
language is en
language is en
language is en
language is en
language is en
language is en
language is en
Not able to detect language
language is en
language is fr
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is pt
language is en
language is en
language is en
language is en
language is pl
language is en
language is en
language is en
language is en
Not able to detect language
language is it
language is en
language is bg
language is en
language is en
language is en
language is en
language is en
language is pt
language is en
language is en
language is en
language is en
language is en
language is en
language is es
language is en
l

language is en
language is en
language is en
language is en
language is en
language is en
language is hr
language is nl
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is de
language is hu
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is es
language is pt
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is es
language is es
language is en
language is en
language is ko
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language is en
language i

In [7]:
texts = []
for text in textsProcessed:
    if len(text)>1:
        texts.append(text)
len(texts)

1807

## Topic Modelling: LDA

In [10]:
dictionary = corpora.Dictionary(texts)

In [14]:
dictionary.token2id

{'C': 0,
 'CO': 1,
 'acidif': 2,
 'activ': 3,
 'address': 4,
 'also': 5,
 'among': 6,
 'anthropogen': 7,
 'articl': 8,
 'avail': 9,
 'beli': 10,
 'caus': 11,
 'chang': 12,
 'climat': 13,
 'concern': 14,
 'consensu': 15,
 'consumpt': 16,
 'continu': 17,
 'contribut': 18,
 'cyclon': 19,
 'degre': 20,
 'develop': 21,
 'drought': 22,
 'earth’': 23,
 'econom': 24,
 'electr': 25,
 'emiss': 26,
 'energi': 27,
 'etc': 28,
 'event': 29,
 'expect': 30,
 'extrem': 31,
 'fire': 32,
 'forest': 33,
 'forum': 34,
 'fuell': 35,
 'global': 36,
 'grid': 37,
 'grow': 38,
 'ice': 39,
 'includ': 40,
 'increas': 41,
 'indian': 42,
 'india’': 43,
 'issu': 44,
 'known': 45,
 'level': 46,
 'life': 47,
 'lifetim': 48,
 'link': 49,
 'live': 50,
 'loss': 51,
 'mean': 52,
 'million': 53,
 'one': 54,
 'particularli': 55,
 'polar': 56,
 'potenti': 57,
 'press': 58,
 'problem': 59,
 'public': 60,
 'read': 61,
 'research': 62,
 'rise': 63,
 'sea': 64,
 'solar': 65,
 'strongli': 66,
 'surfac': 67,
 'temperatur': 68,
 '

In [15]:
corpus = [dictionary.doc2bow(text) for text in textsProcessed]

In [17]:
corpus[1000]

[(4, 1),
 (24, 1),
 (40, 1),
 (44, 1),
 (45, 1),
 (54, 1),
 (97, 1),
 (106, 1),
 (122, 1),
 (124, 1),
 (131, 2),
 (139, 1),
 (143, 4),
 (152, 6),
 (166, 1),
 (168, 1),
 (212, 1),
 (214, 1),
 (215, 1),
 (250, 1),
 (322, 4),
 (342, 2),
 (369, 2),
 (380, 1),
 (408, 1),
 (417, 1),
 (421, 2),
 (500, 1),
 (525, 1),
 (555, 1),
 (557, 1),
 (561, 1),
 (605, 1),
 (630, 1),
 (654, 1),
 (655, 1),
 (688, 1),
 (692, 1),
 (696, 3),
 (698, 2),
 (753, 1),
 (761, 1),
 (765, 1),
 (776, 3),
 (781, 1),
 (807, 1),
 (809, 5),
 (926, 1),
 (929, 3),
 (964, 1),
 (966, 1),
 (1001, 1),
 (1002, 1),
 (1027, 1),
 (1045, 1),
 (1058, 4),
 (1093, 2),
 (1100, 2),
 (1111, 1),
 (1122, 1),
 (1164, 1),
 (1168, 1),
 (1214, 2),
 (1218, 1),
 (1236, 1),
 (1281, 2),
 (1360, 1),
 (1413, 1),
 (1599, 1),
 (1654, 1),
 (1696, 1),
 (1722, 4),
 (1887, 1),
 (1915, 1),
 (1998, 1),
 (2009, 1),
 (2057, 1),
 (2077, 1),
 (2261, 2),
 (2266, 1),
 (2271, 1),
 (2277, 1),
 (2304, 1),
 (2373, 1),
 (2374, 2),
 (2379, 2),
 (2413, 1),
 (2497, 1),
 (2

In [20]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [21]:
ldamodel.print_topics(num_topics=3, num_words=15)

[(0,
  '0.030*"hous" + 0.025*"new" + 0.022*"democrat" + 0.020*"trump" + 0.017*"govern" + 0.014*"pelosi" + 0.013*"speaker" + 0.013*"congress" + 0.011*"—" + 0.011*"senat" + 0.010*"leader" + 0.009*"first" + 0.009*"republican" + 0.009*"demand" + 0.008*"return"'),
 (1,
  '0.008*"said" + 0.008*"climat" + 0.007*"energi" + 0.006*"chang" + 0.006*"new" + 0.005*"year" + 0.005*"power" + 0.004*"state" + 0.004*"project" + 0.004*"work" + 0.004*"global" + 0.003*"report" + 0.003*"would" + 0.003*"peopl" + 0.003*"countri"'),
 (2,
  '0.019*"energi" + 0.010*"compani" + 0.008*"new" + 0.008*"state" + 0.008*"renew" + 0.008*"power" + 0.008*"solar" + 0.007*"electr" + 0.007*"market" + 0.007*"year" + 0.006*"said" + 0.006*"gener" + 0.005*"ga" + 0.005*"tax" + 0.005*"would"')]

In [23]:
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=ldamodel, texts=textsProcessed, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.323863810330071

Coherence Score:  0.5044095077733435


/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


In [26]:
for i in range(len(corpus)):
    tmpTopic = ldamodel[corpus[i]]
    print(f"For doc {i}, corresponding topics & probability are {tmpTopic}")

For doc 0, corresponding topics & probability are [(1, 0.92634904), (2, 0.07039428)]
For doc 1, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 2, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 3, corresponding topics & probability are [(1, 0.11866365), (2, 0.8745628)]
For doc 4, corresponding topics & probability are [(1, 0.20689917), (2, 0.7871028)]
For doc 5, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 6, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 7, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 8, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 9, corresponding topics & probability are [(0, 0.03106367), (1, 0.030752392), (2, 0.9381839)]
For doc 10, corresponding topics 

For doc 448, corresponding topics & probability are [(0, 0.021342536), (1, 0.020309389), (2, 0.9583481)]
For doc 449, corresponding topics & probability are [(0, 0.0128163295), (1, 0.06383481), (2, 0.92334884)]
For doc 450, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 451, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 452, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 453, corresponding topics & probability are [(0, 0.021207975), (2, 0.97354853)]
For doc 454, corresponding topics & probability are [(0, 0.012481382), (1, 0.14801607), (2, 0.8395025)]
For doc 455, corresponding topics & probability are [(2, 0.99203277)]
For doc 456, corresponding topics & probability are [(0, 0.99499124)]
For doc 457, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 458, corresponding topics & proba

For doc 911, corresponding topics & probability are [(1, 0.23849927), (2, 0.759015)]
For doc 912, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 913, corresponding topics & probability are [(0, 0.012641813), (1, 0.013338944), (2, 0.9740192)]
For doc 914, corresponding topics & probability are [(1, 0.1593515), (2, 0.83598626)]
For doc 915, corresponding topics & probability are [(2, 0.9873473)]
For doc 916, corresponding topics & probability are [(1, 0.6076455), (2, 0.3847444)]
For doc 917, corresponding topics & probability are [(1, 0.99353164)]
For doc 918, corresponding topics & probability are [(0, 0.09020633), (1, 0.022843376), (2, 0.8869503)]
For doc 919, corresponding topics & probability are [(2, 0.9953871)]
For doc 920, corresponding topics & probability are [(2, 0.99480426)]
For doc 921, corresponding topics & probability are [(2, 0.9958055)]
For doc 922, corresponding topics & probability are [(1, 0.7248623), (2, 0.27454758)

For doc 1467, corresponding topics & probability are [(0, 0.01951577), (1, 0.023681527), (2, 0.9568027)]
For doc 1468, corresponding topics & probability are [(2, 0.98432434)]
For doc 1469, corresponding topics & probability are [(0, 0.028438136), (1, 0.38923252), (2, 0.58232933)]
For doc 1470, corresponding topics & probability are [(1, 0.6664179), (2, 0.32995495)]
For doc 1471, corresponding topics & probability are [(1, 0.6498319), (2, 0.3463721)]
For doc 1472, corresponding topics & probability are [(1, 0.65409344), (2, 0.34206632)]
For doc 1473, corresponding topics & probability are [(1, 0.013511318), (2, 0.9845063)]
For doc 1474, corresponding topics & probability are [(0, 0.021348324), (1, 0.023380365), (2, 0.9552713)]
For doc 1475, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 1476, corresponding topics & probability are [(1, 0.28122172), (2, 0.71579707)]
For doc 1477, corresponding topics & probability are [(1, 0.1603536), 

For doc 1944, corresponding topics & probability are [(0, 0.12560615), (2, 0.86998445)]
For doc 1945, corresponding topics & probability are [(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]
For doc 1946, corresponding topics & probability are [(0, 0.027702404), (2, 0.9633509)]
For doc 1947, corresponding topics & probability are [(0, 0.043507207), (1, 0.044962876), (2, 0.9115299)]
For doc 1948, corresponding topics & probability are [(1, 0.058831934), (2, 0.93908715)]
For doc 1949, corresponding topics & probability are [(0, 0.065387286), (1, 0.7909421), (2, 0.14367065)]
For doc 1950, corresponding topics & probability are [(0, 0.011423036), (1, 0.97566825), (2, 0.012908735)]
For doc 1951, corresponding topics & probability are [(0, 0.011423034), (1, 0.9756582), (2, 0.0129187945)]
For doc 1952, corresponding topics & probability are [(1, 0.9935321)]
For doc 1953, corresponding topics & probability are [(0, 0.11290585), (1, 0.76024616), (2, 0.12684798)]
For doc 1954, corresponding to

## Vader Sentiment Analysis 

In [13]:
vader = SentimentIntensityAnalyzer()

In [14]:
texts[0]

'January 1, 2019\n\nAbout a month and a half ago, I attended the Global Energy Forum at Stanford University in the USA. Among the many prominent speakers there were Microsoft’s Bill Gates, former US Secretary of State George Shultz and California’s Governor Jerry Brown. The forum addressed one of the most pressing issues of our lifetimes - global energy and climate change. I would like to present some of the things I learnt at the forum along with India’s role in the global energy playground.\n\nEnergy is a direct measure of prosperity and standard of living. Everything that makes your life comfortable needs energy - transportation, almost everything in your house from your AC to your fridge, industrial processes that manufacture everything you use from plastics to metal to furniture, and so on. The World Bank’s report on World Development Indicators [1] clearly shows that the most developed countries have a much higher per capita energy consumption than the least developed countries. 

In [15]:
vader.polarity_scores(texts[0])

{'neg': 0.033, 'neu': 0.829, 'pos': 0.138, 'compound': 0.9985}

In [16]:
for i in range(len(texts)):
    sentiment = vader.polarity_scores(texts[i])
    print(f"For string {i}, sentiment sores: {sentiment}")

For string 0, sentiment sores: {'neg': 0.033, 'neu': 0.829, 'pos': 0.138, 'compound': 0.9985}
For string 1, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 2, sentiment sores: {'neg': 0.0, 'neu': 0.996, 'pos': 0.004, 'compound': 0.34}
For string 3, sentiment sores: {'neg': 0.007, 'neu': 0.819, 'pos': 0.174, 'compound': 0.9925}
For string 4, sentiment sores: {'neg': 0.024, 'neu': 0.82, 'pos': 0.156, 'compound': 0.9955}
For string 5, sentiment sores: {'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.8478}
For string 6, sentiment sores: {'neg': 0.132, 'neu': 0.814, 'pos': 0.054, 'compound': -0.6808}
For string 7, sentiment sores: {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
For string 8, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 9, sentiment sores: {'neg': 0.0, 'neu': 0.923, 'pos': 0.077, 'compound': 0.8479}
For string 10, sentiment sores: {'neg': 0.012, 'neu': 0.898, 'pos': 0.09, 'compound': 0.9656}
F

For string 92, sentiment sores: {'neg': 0.024, 'neu': 0.877, 'pos': 0.099, 'compound': 0.9968}
For string 93, sentiment sores: {'neg': 0.057, 'neu': 0.822, 'pos': 0.122, 'compound': 0.9966}
For string 94, sentiment sores: {'neg': 0.059, 'neu': 0.84, 'pos': 0.101, 'compound': 0.659}
For string 95, sentiment sores: {'neg': 0.0, 'neu': 0.848, 'pos': 0.152, 'compound': 0.9855}
For string 96, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 97, sentiment sores: {'neg': 0.006, 'neu': 0.959, 'pos': 0.035, 'compound': 0.9333}
For string 98, sentiment sores: {'neg': 0.034, 'neu': 0.962, 'pos': 0.004, 'compound': -0.9623}
For string 99, sentiment sores: {'neg': 0.007, 'neu': 0.97, 'pos': 0.023, 'compound': 0.5994}
For string 100, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 101, sentiment sores: {'neg': 0.007, 'neu': 0.97, 'pos': 0.023, 'compound': 0.5994}
For string 102, sentiment sores: {'neg': 0.007, 'neu': 0.97, 'pos': 0.0

For string 181, sentiment sores: {'neg': 0.009, 'neu': 0.847, 'pos': 0.144, 'compound': 0.9999}
For string 182, sentiment sores: {'neg': 0.064, 'neu': 0.852, 'pos': 0.083, 'compound': 0.9455}
For string 183, sentiment sores: {'neg': 0.064, 'neu': 0.852, 'pos': 0.083, 'compound': 0.9455}
For string 184, sentiment sores: {'neg': 0.064, 'neu': 0.852, 'pos': 0.083, 'compound': 0.9455}
For string 185, sentiment sores: {'neg': 0.063, 'neu': 0.854, 'pos': 0.084, 'compound': 0.9571}
For string 186, sentiment sores: {'neg': 0.003, 'neu': 0.909, 'pos': 0.088, 'compound': 0.9964}
For string 187, sentiment sores: {'neg': 0.037, 'neu': 0.901, 'pos': 0.062, 'compound': 0.3818}
For string 188, sentiment sores: {'neg': 0.0, 'neu': 0.871, 'pos': 0.129, 'compound': 0.933}
For string 189, sentiment sores: {'neg': 0.007, 'neu': 0.985, 'pos': 0.009, 'compound': 0.4215}
For string 190, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 191, sentiment sores: {'neg': 0.0, 'neu':

For string 274, sentiment sores: {'neg': 0.189, 'neu': 0.713, 'pos': 0.097, 'compound': -0.9994}
For string 275, sentiment sores: {'neg': 0.0, 'neu': 0.876, 'pos': 0.124, 'compound': 0.9764}
For string 276, sentiment sores: {'neg': 0.037, 'neu': 0.815, 'pos': 0.148, 'compound': 0.9993}
For string 277, sentiment sores: {'neg': 0.15, 'neu': 0.749, 'pos': 0.101, 'compound': -0.4019}
For string 278, sentiment sores: {'neg': 0.0, 'neu': 0.862, 'pos': 0.138, 'compound': 0.9883}
For string 279, sentiment sores: {'neg': 0.008, 'neu': 0.843, 'pos': 0.15, 'compound': 0.9868}
For string 280, sentiment sores: {'neg': 0.067, 'neu': 0.763, 'pos': 0.17, 'compound': 0.9956}
For string 281, sentiment sores: {'neg': 0.053, 'neu': 0.868, 'pos': 0.078, 'compound': 0.5838}
For string 282, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 283, sentiment sores: {'neg': 0.015, 'neu': 0.918, 'pos': 0.067, 'compound': 0.9459}
For string 284, sentiment sores: {'neg': 0.036, 'neu':

For string 365, sentiment sores: {'neg': 0.061, 'neu': 0.787, 'pos': 0.151, 'compound': 0.999}
For string 366, sentiment sores: {'neg': 0.031, 'neu': 0.852, 'pos': 0.117, 'compound': 0.9856}
For string 367, sentiment sores: {'neg': 0.01, 'neu': 0.896, 'pos': 0.094, 'compound': 0.984}
For string 368, sentiment sores: {'neg': 0.008, 'neu': 0.953, 'pos': 0.039, 'compound': 0.9806}
For string 369, sentiment sores: {'neg': 0.0, 'neu': 0.806, 'pos': 0.194, 'compound': 0.802}
For string 370, sentiment sores: {'neg': 0.0, 'neu': 0.848, 'pos': 0.152, 'compound': 0.9923}
For string 371, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 372, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 373, sentiment sores: {'neg': 0.0, 'neu': 0.992, 'pos': 0.008, 'compound': 0.2732}
For string 374, sentiment sores: {'neg': 0.0, 'neu': 0.992, 'pos': 0.008, 'compound': 0.2732}
For string 375, sentiment sores: {'neg': 0.019, 'neu': 0.959, 'pos': 0

...

For string 1883, sentiment sores: {'neg': 0.02, 'neu': 0.869, 'pos': 0.111, 'compound': 0.9965}
For string 1884, sentiment sores: {'neg': 0.008, 'neu': 0.854, 'pos': 0.138, 'compound': 0.9823}
For string 1885, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 1886, sentiment sores: {'neg': 0.029, 'neu': 0.862, 'pos': 0.109, 'compound': 0.8489}
For string 1887, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 1888, sentiment sores: {'neg': 0.0, 'neu': 0.96, 'pos': 0.04, 'compound': 0.7506}
For string 1889, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 1890, sentiment sores: {'neg': 0.029, 'neu': 0.86, 'pos': 0.111, 'compound': 0.8489}
For string 1891, sentiment sores: {'neg': 0.073, 'neu': 0.79, 'pos': 0.137, 'compound': 0.9981}
For string 1892, sentiment sores: {'neg': 0.071, 'neu': 0.847, 'pos': 0.082, 'compound': 0.1007}
For string 1893, sentiment sores: {'neg': 0.039, 'neu': 0.93, 'p

For string 1978, sentiment sores: {'neg': 0.016, 'neu': 0.97, 'pos': 0.013, 'compound': -0.6705}
For string 1979, sentiment sores: {'neg': 0.016, 'neu': 0.97, 'pos': 0.013, 'compound': -0.6705}
For string 1980, sentiment sores: {'neg': 0.032, 'neu': 0.871, 'pos': 0.097, 'compound': 0.9978}
For string 1981, sentiment sores: {'neg': 0.066, 'neu': 0.866, 'pos': 0.068, 'compound': -0.8175}
For string 1982, sentiment sores: {'neg': 0.053, 'neu': 0.846, 'pos': 0.101, 'compound': 0.9569}
For string 1983, sentiment sores: {'neg': 0.022, 'neu': 0.812, 'pos': 0.166, 'compound': 0.9983}
For string 1984, sentiment sores: {'neg': 0.059, 'neu': 0.782, 'pos': 0.159, 'compound': 0.9099}
For string 1985, sentiment sores: {'neg': 0.025, 'neu': 0.918, 'pos': 0.057, 'compound': 0.9538}
For string 1986, sentiment sores: {'neg': 0.051, 'neu': 0.949, 'pos': 0.0, 'compound': -0.0772}
For string 1987, sentiment sores: {'neg': 0.083, 'neu': 0.855, 'pos': 0.062, 'compound': -0.9976}
For string 1988, sentiment so

For string 2066, sentiment sores: {'neg': 0.041, 'neu': 0.894, 'pos': 0.065, 'compound': 0.8504}
For string 2067, sentiment sores: {'neg': 0.007, 'neu': 0.984, 'pos': 0.008, 'compound': 0.1779}
For string 2068, sentiment sores: {'neg': 0.027, 'neu': 0.902, 'pos': 0.071, 'compound': 0.9819}
For string 2069, sentiment sores: {'neg': 0.035, 'neu': 0.783, 'pos': 0.182, 'compound': 0.9995}
For string 2070, sentiment sores: {'neg': 0.041, 'neu': 0.85, 'pos': 0.11, 'compound': 0.9935}
For string 2071, sentiment sores: {'neg': 0.013, 'neu': 0.977, 'pos': 0.01, 'compound': -0.264}
For string 2072, sentiment sores: {'neg': 0.037, 'neu': 0.954, 'pos': 0.009, 'compound': -0.9231}
For string 2073, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 2074, sentiment sores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
For string 2075, sentiment sores: {'neg': 0.057, 'neu': 0.818, 'pos': 0.125, 'compound': 0.8399}
For string 2076, sentiment sores: {'neg': 0.007, '

For string 2154, sentiment sores: {'neg': 0.063, 'neu': 0.83, 'pos': 0.107, 'compound': 0.9989}
For string 2155, sentiment sores: {'neg': 0.0, 'neu': 0.76, 'pos': 0.24, 'compound': 0.7184}
For string 2156, sentiment sores: {'neg': 0.012, 'neu': 0.871, 'pos': 0.116, 'compound': 0.9896}
For string 2157, sentiment sores: {'neg': 0.012, 'neu': 0.868, 'pos': 0.12, 'compound': 0.9908}
For string 2158, sentiment sores: {'neg': 0.026, 'neu': 0.871, 'pos': 0.102, 'compound': 0.999}
For string 2159, sentiment sores: {'neg': 0.025, 'neu': 0.851, 'pos': 0.124, 'compound': 0.9918}
For string 2160, sentiment sores: {'neg': 0.048, 'neu': 0.786, 'pos': 0.166, 'compound': 0.999}
For string 2161, sentiment sores: {'neg': 0.152, 'neu': 0.848, 'pos': 0.0, 'compound': -0.9927}
For string 2162, sentiment sores: {'neg': 0.005, 'neu': 0.975, 'pos': 0.02, 'compound': 0.7579}
For string 2163, sentiment sores: {'neg': 0.032, 'neu': 0.857, 'pos': 0.111, 'compound': 0.9994}
For string 2164, sentiment sores: {'neg'

For string 2242, sentiment sores: {'neg': 0.022, 'neu': 0.855, 'pos': 0.123, 'compound': 0.9973}
For string 2243, sentiment sores: {'neg': 0.0, 'neu': 0.863, 'pos': 0.137, 'compound': 0.9957}
For string 2244, sentiment sores: {'neg': 0.008, 'neu': 0.983, 'pos': 0.01, 'compound': 0.2168}
For string 2245, sentiment sores: {'neg': 0.073, 'neu': 0.825, 'pos': 0.101, 'compound': 0.9966}
For string 2246, sentiment sores: {'neg': 0.029, 'neu': 0.833, 'pos': 0.138, 'compound': 0.9139}
For string 2247, sentiment sores: {'neg': 0.0, 'neu': 0.886, 'pos': 0.114, 'compound': 0.9173}
For string 2248, sentiment sores: {'neg': 0.017, 'neu': 0.892, 'pos': 0.091, 'compound': 0.9945}
For string 2249, sentiment sores: {'neg': 0.063, 'neu': 0.829, 'pos': 0.108, 'compound': 0.659}
For string 2250, sentiment sores: {'neg': 0.071, 'neu': 0.847, 'pos': 0.082, 'compound': 0.1007}
For string 2251, sentiment sores: {'neg': 0.04, 'neu': 0.909, 'pos': 0.051, 'compound': 0.6908}
For string 2252, sentiment sores: {'n

## neural net on sentiment analysis (text classification) 

In [2]:

df = pd.read_csv('/Users/sizhenhan/Downloads/NLP-analysis-of-Energy-sector-based-on-Gdelt-master/data/twitter-airline-sentiment/tweets.csv',usecols=['text', 'airline_sentiment'])


In [3]:
df.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [4]:
df.text[3]

'@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse'

In [5]:
airline = ProcessPipeline()

In [6]:
df.text = list(map(lambda x:airline.pre_process(x,return_str=True,steps=['remove_digits','remove_stopwords','remove_punctuation']),df.text))



In [7]:
df.head()

,airline_sentiment,text
0,neutral,VirginAmerica dhepburn said
1,positive,VirginAmerica plus youve added commercials exp...
2,neutral,VirginAmerica didnt today Must mean need take ...
3,negative,VirginAmerica really aggressive blast obnoxiou...
4,negative,VirginAmerica really big bad thing


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.airline_sentiment, test_size=0.1, random_state=2019)

In [11]:
seq_lengths = X_train.apply(lambda x: len(x.split(' ')))
seq_lengths.describe()

count    13176.000000
mean        10.235656
std          3.997697
min          1.000000
25%          7.000000
50%         11.000000
75%         13.000000
max         22.000000
Name: text, dtype: float64

In [84]:
tk = Tokenizer(num_words=10000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
tk.fit_on_texts(X_train)

In [85]:
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

In [98]:
tk.word_index

{'united': 1,
 'flight': 2,
 'usairways': 3,
 'americanair': 4,
 'southwestair': 5,
 'jetblue': 6,
 'not': 7,
 'no': 8,
 'get': 9,
 'thanks': 10,
 'cancelled': 11,
 'service': 12,
 'help': 13,
 'time': 14,
 'customer': 15,
 'im': 16,
 'us': 17,
 'hours': 18,
 'flights': 19,
 'amp': 20,
 'hold': 21,
 'plane': 22,
 'thank': 23,
 'cant': 24,
 'still': 25,
 'one': 26,
 'need': 27,
 'please': 28,
 'would': 29,
 'delayed': 30,
 'flightled': 31,
 'virginamerica': 32,
 'back': 33,
 'gate': 34,
 'call': 35,
 'dont': 36,
 'bag': 37,
 'hour': 38,
 'got': 39,
 'late': 40,
 'phone': 41,
 'like': 42,
 'today': 43,
 'airline': 44,
 'guys': 45,
 'waiting': 46,
 'know': 47,
 'fly': 48,
 'airport': 49,
 'day': 50,
 'trying': 51,
 'way': 52,
 'going': 53,
 'u': 54,
 'hrs': 55,
 'ive': 56,
 'never': 57,
 'great': 58,
 'change': 59,
 'wait': 60,
 'last': 61,
 'make': 62,
 'flying': 63,
 'really': 64,
 'tomorrow': 65,
 'go': 66,
 'minutes': 67,
 'weather': 68,
 'even': 69,
 'delay': 70,
 'home': 71,
 'good'

In [90]:
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=24,padding = 'post')
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen= 24, padding = 'post')

In [103]:
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)

In [104]:
y_train_oh

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [112]:
_input = Input(shape=(24,))
embed = Embedding(10000,200)(_input)
flat = Flatten()(embed)
dense = Dense(3,activation = 'softmax')(flat)
emb_model = Model(_input,dense)
emb_model.summary()
emb_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 24)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 24, 200)           2000000   
_________________________________________________________________
flatten_3 (Flatten)          (None, 4800)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 14403     
Total params: 2,014,403
Trainable params: 2,014,403
Non-trainable params: 0
_________________________________________________________________


In [113]:

emb_model.fit(X_train_seq_trunc, y_train_oh, epochs=10, batch_size=32, verbose=1)


W0827 20:38:48.108190 4522026432 deprecation.py:323] From /Users/sizhenhan/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 13176 samples
Epoch 1/10
13176/13176 [==============================] - 5s 396us/sample - loss: 0.6488 - accuracy: 0.7343
Epoch 2/10
13176/13176 [==============================] - 5s 359us/sample - loss: 0.3490 - accuracy: 0.8727
Epoch 3/10
13176/13176 [==============================] - 5s 358us/sample - loss: 0.1821 - accuracy: 0.9424
Epoch 4/10
13176/13176 [==============================] - 5s 365us/sample - loss: 0.1005 - accuracy: 0.9715
Epoch 5/10
13176/13176 [==============================] - 5s 364us/sample - loss: 0.0635 - accuracy: 0.9839
Epoch 6/10
13176/13176 [==============================] - 5s 365us/sample - loss: 0.0476 - accuracy: 0.9871
Epoch 7/10
13176/13176 [==============================] - 5s 361us/sample - loss: 0.0391 - accuracy: 0.9904
Epoch 8/10
13176/13176 [==============================] - 5s 364us/sample - loss: 0.0332 - accuracy: 0.9915
Epoch 9/10
13176/13176 [==============================] - 5s 367us/sample - loss: 0.0299 - accuracy: 0.9917
Epoch

In [121]:
emb_y = emb_model.predict(X_test_seq_trunc)


In [120]:
pred = []
for value in emb_y:
    if max(value) == value[0]:
        pred.append(0)
    elif max(value) == value[1]:
        pred.append(1)
    else:
        pred.append(2)

In [123]:
accuracy_score(pred,y_test_le)

0.7800546448087432

In [131]:
!ls lib

glove.6B.100d.txt glove.6B.200d.txt glove.6B.300d.txt glove.6B.50d.txt


In [127]:
urllib.request.urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", "gloveEmb.zip")

('gloveEmb.zip', <http.client.HTTPMessage at 0x1a406746a0>)

In [130]:
zip_ref = zipfile.ZipFile("gloveEmb.zip", 'r') 
zip_ref.extractall("lib") 
zip_ref.close()

In [132]:
glove_file = 'glove.6B.' + str(200) + 'd.txt'
emb_dict = {}
glove = open("lib/" + glove_file)

In [137]:
glove_file = 'glove.6B.' + str(200) + 'd.txt'
emb_dict = {}
glove = open("lib/" + glove_file)
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    emb_dict[word] = vector
glove.close()

In [143]:
emb_matrix = np.zeros((10000, 200))
for w, i in tk.word_index.items():
    if i < 10000:
        vect = emb_dict.get(w)
        if vect is not None:
            emb_matrix[i] = vect
    else:
        break

In [146]:
_input = Input(shape=(24,))
embed = Embedding(10000,200)(_input)
flat = Flatten()(embed)
dense = Dense(3,activation = 'softmax')(flat)
glove_model = Model(_input,dense)
glove_model.layers[1].set_weights([emb_matrix])
glove_model.layers[1].trainable = True
glove_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [147]:
glove_model.fit(X_train_seq_trunc, y_train_oh, epochs=10, batch_size=32, verbose=1)



Train on 13176 samples
Epoch 1/10
13176/13176 [==============================] - 5s 391us/sample - loss: 0.6295 - accuracy: 0.7469
Epoch 2/10
13176/13176 [==============================] - 5s 383us/sample - loss: 0.3988 - accuracy: 0.8484
Epoch 3/10
13176/13176 [==============================] - 5s 377us/sample - loss: 0.2678 - accuracy: 0.9060
Epoch 4/10
13176/13176 [==============================] - 5s 393us/sample - loss: 0.1750 - accuracy: 0.9454
Epoch 5/10
13176/13176 [==============================] - 5s 370us/sample - loss: 0.1180 - accuracy: 0.9674
Epoch 6/10
13176/13176 [==============================] - 5s 390us/sample - loss: 0.0852 - accuracy: 0.9782
Epoch 7/10
13176/13176 [==============================] - 5s 380us/sample - loss: 0.0635 - accuracy: 0.9850
Epoch 8/10
13176/13176 [==============================] - 5s 380us/sample - loss: 0.0516 - accuracy: 0.9882
Epoch 9/10
13176/13176 [==============================] - 5s 379us/sample - loss: 0.0430 - accuracy: 0.9892
Epoch

In [148]:
glove_y = glove_model.predict(X_test_seq_trunc)
pred2 = []
for value in glove_y:
    if max(value) == value[0]:
        pred2.append(0)
    elif max(value) == value[1]:
        pred2.append(1)
    else:
        pred2.append(2)

In [149]:
accuracy_score(pred2,y_test_le)

0.7691256830601093

In [150]:
texts = [
    "The cat sat on the hat",
    "The dog ate the cat and the hat",
    "Dog and cat are good friends"
]

In [155]:
tf = CountVectorizer()
featuresTf = tf.fit_transform(texts)
pd.DataFrame(featuresTf.todense(),columns=tf.get_feature_names())

,and,are,ate,cat,dog,friends,good,hat,on,sat,the
0,0,0,0,1,0,0,0,1,1,1,2
1,1,0,1,1,1,0,0,1,0,0,3
2,1,1,0,1,1,1,1,0,0,0,0


In [156]:
tfidf = TfidfVectorizer()
features = tfidf.fit_transform(texts)
pd.DataFrame(features.todense(),columns=tfidf.get_feature_names())

,and,are,ate,cat,dog,friends,good,hat,on,sat,the
0,0.000000,0.00000,0.000000,0.257992,0.000000,0.00000,0.00000,0.332211,0.436818,0.436818,0.664422
1,0.264148,0.00000,0.347322,0.205134,0.264148,0.00000,0.00000,0.264148,0.000000,0.000000,0.792443
2,0.358291,0.47111,0.000000,0.278245,0.358291,0.47111,0.47111,0.000000,0.000000,0.000000,0.000000


In [157]:
texts[0]

'The cat sat on the hat'

In [159]:
with open('texts_raw_new.pickle', 'rb') as handle:
    texts = pickle.load(handle)

In [161]:
pipeline = ProcessPipeline(texts)

textsProcessed = pipeline.run()

language is en
language is en


W0828 14:57:54.025772 4522026432 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0828 14:57:54.028450 4522026432 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W0828 14:57:54.030699 4522026432 summarizer.py:370] Please add more sentences to the text. The number of reachable nodes is below 3
W0828 14:57:54.033324 4522026432 summarizer.py:440] Couldn't get relevant sentences.


language is de
language is en


W0828 14:57:54.102608 4522026432 summarizer.py:423] Input text is empty.


language is bg


...

W0828 14:59:08.215002 4522026432 summarizer.py:423] Input text is empty.


language is ar


W0828 14:59:08.221100 4522026432 summarizer.py:423] Input text is empty.


language is en


W0828 14:59:08.249582 4522026432 summarizer.py:432] Input text is expected to have at least 10 sentences.
W0828 14:59:08.251312 4522026432 summarizer.py:356] Input corpus is expected to have at least 10 documents.


In [162]:
texts = []
for text in textsProcessed:
    if len(text)>1:
        texts.append(text)
len(texts)

1807

In [164]:
model = gensim.models.Word2Vec(texts, min_count=1,size=300,workers=4)

In [165]:
model.most_similar("trump")

/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('your', 0.9548013806343079),
 ('hire', 0.9538844227790833),
 ('cabinet', 0.9506486654281616),
 ('isnt', 0.9453733563423157),
 ('trump’', 0.9413491487503052),
 ('reinc', 0.9383353590965271),
 ('showcas', 0.928868293762207),
 ('unelect', 0.9232478737831116),
 ('impeach', 0.9159296154975891),
 ('you’', 0.9154974818229675)]

In [166]:
model.similarity('stock', 'share')

/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.9760872

In [167]:
model.most_similar("stock")

/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('managers–kelli', 0.9857401847839355),
 ('relev', 0.9848661422729492),
 ('must', 0.9833390116691589),
 ('tree', 0.9829999208450317),
 ('arrog', 0.9828389883041382),
 ('ten', 0.9828318357467651),
 ('harvest', 0.9822014570236206),
 ('sverig', 0.981253981590271),
 ('forum', 0.9804190397262573),
 ('juri', 0.9801247119903564)]

In [168]:
texts[0][:10]

['forum',
 'address',
 'one',
 'press',
 'issu',
 'lifetim',
 'global',
 'energi',
 'climat',
 'chang']